In [1]:
from glob import  glob
from pathlib import Path

import numpy

from become_yukarin.config.config import create_from_json as create_config
from become_yukarin.config.sr_config import create_from_json as create_sr_config
from become_yukarin.dataset.dataset import AcousticFeatureProcess
from become_yukarin.dataset.dataset import WaveFileLoadProcess
from become_yukarin.param import Param
from become_yukarin import SuperResolution
from become_yukarin import AcousticConverter

from IPython.display import Audio

# loading model1
model_path = Path('./dat/model/yukari_1st/predictor_2275000.npz')
config_path = Path('./dat/model/yukari_1st/config.json')
config = create_config(config_path)
voice_changer = AcousticConverter(config, model_path, gpu=0)

# loading model2
model_path = Path('./dat/model/yukari_2nd/predictor_120000.npz')
config_path = Path('./dat/model/yukari_2nd/config.json')
sr_config = create_sr_config(config_path)
super_resolution = SuperResolution(sr_config, model_path, gpu=0)

In [2]:
import pyaudio
import wave


# recoding process
rec_time = 5 # 録音時間
file_path = "test_data/output.wav" # 音声を保存するファイル名
fmt = pyaudio.paInt16 # 音声のフォーマット
ch = 1 # チャンネル数
sampling_rate = 22050 # サンプリング周波数
chunk = 2**11 # チャンク（データ点数）
audio = pyaudio.PyAudio()
index = 26 # 録音デバイスのインデックス番号

print("recoding start ...")

stream = audio.open(format=fmt, channels=ch, rate=sampling_rate, input=True,
               input_device_index = index, frames_per_buffer=chunk)
# 録音処理
frames = []
for i in range(0, int(sampling_rate / chunk * rec_time)):
    data = stream.read(chunk)
    frames.append(data)

print("recoding end...")

# 録音終了処理
stream.stop_stream()
stream.close()
audio.terminate()

# 録音データをファイルに保存
wav = wave.open(file_path, 'wb')
wav.setnchannels(ch)
wav.setsampwidth(audio.get_sample_size(fmt))
wav.setframerate(sampling_rate)
wav.writeframes(b''.join(frames))
wav.close()

# test voice conversion
p = './test_data/output.wav'
wave_in = voice_changer._wave_process(str(p))
f_in = voice_changer._feature_process(wave_in)
f_low = voice_changer.convert_to_feature(f_in)
s_high = super_resolution.convert(f_low.spectrogram.astype(numpy.float32))

# make audio
rate = sr_config.dataset.param.voice_param.sample_rate
wave = super_resolution(s_high, acoustic_feature=f_low, sampling_rate=rate)
Audio(data=wave.wave , rate=rate)

recoding start ...
recoding end...


In [12]:
wave

Wave(wave=array([-0.00014989, -0.0001286 , -0.00012303, ..., -0.00199063,
       -0.00174656, -0.00147358]), sampling_rate=22050)

In [17]:
audio

In [11]:
rate

22050